In [1]:
# import sys
# import os
# import pandas as pd
# import numpy as np
# import csv
# from sklearn.model_selection import train_test_split

# print('...Start splitting data')
# PROJECT_DIR = os.path.join(os.pardir, os.path.dirname(os.path.dirname(__file__)))
# DATA_DIR = os.path.join(PROJECT_DIR, 'sensitive_data')
# input_file_path = os.path.join(DATA_DIR, 'chf_deid_annotated_gold_origin.csv')
# output_file_path_train = os.path.join(DATA_DIR, 'chf_train_gold_set.csv')
# output_file_path_test = os.path.join(DATA_DIR, 'chf_test_gold_set.csv')

# gold_standard = pd.read_csv(input_file_path, header=0, index_col=0)
# gold_standard = gold_standard[gold_standard["label"] != -1]
# train, test = train_test_split(gold_standard, test_size=0.3)
# train.reset_index().to_csv(output_file_path_train, index=False)
# test.reset_index().to_csv(output_file_path_test, index=False)
# print("Train neg: ", np.sum(train["label"] == '0'))
# print("Train pos: ", np.sum(train["label"] == '1'))
# print("Test neg: ", np.sum(test["label"] == '0'))
# print("Test pos: ", np.sum(test["label"] == '1'))

In [2]:
import os
from os import listdir
from os.path import isfile, join
import glob
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2

import requests
import json
import time
import itertools
import os
import tmdbsimple as tmdb
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd
import csv

In [41]:
project_path = os.path.abspath(os.path.join('__file__' ,"../.."))
data_dir = os.path.join(project_path, 'Data')
# file_path = os.path.join(data_dir, 'tmdb-5000-movie-dataset/tmdb_5000_movies.csv')
file_path = os.path.join(data_dir, 'genre_binary_list.csv')
cols = ["id_x", "Drama"]
data = pd.read_csv(file_path, header=0, usecols=cols)

In [42]:
movie_posters_path = os.path.abspath(os.path.join('__file__' ,"../posters_final_with_id"))

In [43]:
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [66]:
missing_genres_ids = [191229, 346081, 371085, 48382, 325140, 357834, 137955, 206412, 219716, 335874, 279759, 331493, 294550, 380097, 297100, 325579, 198370, 328307, 281189, 162396, 365052, 300327, 320435, 194588, 176074, 282128, 38786, 126186]

In [86]:
data_arr = []
from PIL import Image
for poster_path in listdir_nohidden(movie_posters_path):
    poster_name = poster_path[89:]
    poster_id = int(poster_name[2:-4])
    if poster_id in missing_genres_ids:
        continue
    movie_elem = data.loc[data['id_x'] == poster_id]
    drama_label = int(movie_elem["Drama"].values[-1])
    data_arr.append((poster_id, drama_label))

In [87]:
random.shuffle(data_arr)
print(len(data_arr))
num_train = int(len(data_arr) * 0.9)
num_test = len(data_arr) - num_train
print("train: {}, test: {}".format(num_train, num_test))
train_data = data_arr[:num_train]
test_data = data_arr[num_train:]
print(len(train_data), len(test_data))

4775
train: 4297, test: 478
4297 478


In [90]:
print("train positives: {}".format(np.sum([example[1] for example in train_data])))
print("test_positives: {}".format(np.sum([example[1] for example in test_data])))

train positives: 2076
test_positives: 221


In [93]:
train_positive_path = os.path.abspath(os.path.join('__file__' ,"../train/1"))
train_negative_path = os.path.abspath(os.path.join('__file__' ,"../train/0"))
test_positive_path = os.path.abspath(os.path.join('__file__' ,"../test/1"))
test_negative_path = os.path.abspath(os.path.join('__file__' ,"../test/0"))
print("train pos: {}, \ntrain neg: {}, \ntest pos: {}, \ntest neg: {}".format(train_positive_path, train_negative_path, test_positive_path, test_negative_path))

train pos: /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1, 
train neg: /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0, 
test pos: /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/1, 
test neg: /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/0


In [109]:
train_pos_ids = [example[0] for example in train_data if example[1] == 1]
train_neg_ids = [example[0] for example in train_data if example[1] == 0]
test_pos_ids = [example[0] for example in test_data if example[1] == 1]
test_neg_ids = [example[0] for example in test_data if example[1] == 0]

In [111]:
from shutil import copyfile

for poster_path in listdir_nohidden(movie_posters_path):
    poster_name = poster_path[89:]
    poster_id = int(poster_name[2:-4])
    new_file_path = None
    if poster_id in missing_genres_ids:
        continue
    # Positive training example
    if poster_id in train_pos_ids:
        new_file_path = train_positive_path
    # Negative training example
    elif poster_id in train_neg_ids:
        new_file_path = train_negative_path
    # Positive test example
    elif poster_id in test_pos_ids:
        new_file_path = test_positive_path
    # Negative test example
    elif poster_id in test_neg_ids:
        new_file_path = test_negative_path
    
    print(poster_id, new_file_path)
    if new_file_path is not None:
        new_file_path = os.path.join(new_file_path, poster_name)
        copyfile(poster_path, new_file_path)

26963 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
1266 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
13700 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/0
13066 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
14709 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
2009 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
30867 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
46729 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
11665 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/0
13072 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
1272 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
15365 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
23827 /Users/amitschechter/Deskto

26618 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
9799 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
8487 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
13853 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/1
7944 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/0
9969 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
2196 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
36351 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/1
20533 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
226 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/1
540 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
227 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
5820 /Users/amitschechter/Desktop/Spring18/

12437 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
16651 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
10220 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/1
2752 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
9607 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
2034 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
53457 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
18126 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
57157 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
1273 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
91070 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
244776 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
10208 /Users/amitschechter/Deskt

17820 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
10197 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
102362 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
24226 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
17834 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
10183 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
755 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
49948 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
29064 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
1852 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
9981 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
9759 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
3512 /Users/amitschechter/Deskto

1927 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
146 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
20653 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
152 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/0
109424 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
12193 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
1933 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
14284 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
48289 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
634 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
104896 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
8095 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
109418 /Users/amitschechter/Desktop/

...

80215 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
6106 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
1669 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
19150 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
14474 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
12763 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
46332 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
36825 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/0
26388 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/1
256740 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
79698 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
8665 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
217708 /Users/amitschechter/Desk

9543 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/0
6116 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
54054 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
2370 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
258755 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
9557 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
8649 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/1
1123 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
7552 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
18276 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
1645 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
7220 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
1651 /Users/amitschechter/Desktop/Spri

19419 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
1452 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
24940 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
79 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
125263 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
15237 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
18713 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
100975 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
4258 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
11051 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
9754 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
89750 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
292539 /Users/amitschechter/Desk

10588 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
37137 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
21755 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
287903 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/0
26748 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
3482 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
252680 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
329540 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
14275 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
42033 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
2575 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
10013 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
12162 /Users/amitschechter/D

10364 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
37903 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
10833 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
3093 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
38167 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
61012 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
22881 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
37532 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
19803 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
10199 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
11287 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0
13490 /Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1
1690 /Users/amitschechter/Des

In [115]:
from PIL import Image
import numpy as np
for path in ['/Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/1',
            '/Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/train/0',
            '/Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/1',
            '/Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/test/0']
for poster_path in listdir_nohidden(movie_posters_path):
    i = Image.open(poster_path)
    iar = np.asarray(i)
    if iar.shape[2] != 3:
        print(poster_path)

/Users/amitschechter/Desktop/Spring18/CS231N/Project/movie-posters/posters_final_with_id


IndexError: tuple index out of range